In [ ]:
!pip install pandas
!pip install numpy
!pip install openpyxl
!pip install lifetimes

  Using cached scipy-1.14.1-cp310-cp310-win_amd64.whl (44.8 MB)


You should consider upgrading via the 'C:\Users\91992\Desktop\project\RFM_Analysis\RFMvenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [63]:
df = pd.read_excel("dataset/online_retail_II.xlsx","Year 2010-2011")

# df = pd.read_excel("dataset/online_retail_II.xlsx", "Year 2009-2010", nrows=600000)   

In [64]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [65]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [66]:
df = df.dropna()
# df.dropna(inplace = True)

In [67]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, 'Price')
replace_with_thresholds(df, 'Quantity')



In [68]:
df.isnull().sum()


Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [69]:
df = df[~df["Invoice"].str.contains("C", na = False)]


In [70]:
first_date = df["InvoiceDate"].min()
last_date = df["InvoiceDate"].max()
print(f'earliest date: {first_date}')
print(f'last date: {last_date}')


earliest date: 2010-12-01 08:26:00
last date: 2011-12-09 12:50:00


In [71]:
recency = (last_date - df.groupby("Customer ID").agg({"InvoiceDate":"max"})).rename(columns={"InvoiceDate": "recency"})
recency.head()

,recency
Customer ID,
12346.0,325 days 02:49:00
12347.0,1 days 20:58:00
12348.0,74 days 23:37:00
12349.0,18 days 02:59:00
12350.0,309 days 20:49:00


In [72]:
frequency = df.groupby("Customer ID").agg({"InvoiceDate":"nunique"}).rename(columns={"InvoiceDate": "Frequency"})

frequency.head()

,Frequency
Customer ID,
12346.0,1
12347.0,7
12348.0,4
12349.0,1
12350.0,1


In [73]:
df["TotalPrice"] = df["Quantity"] * df["Price"]
monetary = df.groupby("Customer ID").agg({"TotalPrice":"sum"}).rename(columns={"TotalPrice":"monetary"})
monetary.head()

,monetary
Customer ID,
12346.0,315.120
12347.0,4310.000
12348.0,1771.905
12349.0,1491.970
12350.0,331.585


In [74]:
T = (last_date - df.groupby("Customer ID").agg({"InvoiceDate":"min"})).rename(columns={"InvoiceDate": "T"})
T.head()

,T
Customer ID,
12346.0,325 days 02:49:00
12347.0,366 days 21:53:00
12348.0,357 days 17:41:00
12349.0,18 days 02:59:00
12350.0,309 days 20:49:00


In [193]:
rfm = pd.concat([recency, frequency, monetary, T], axis=1)
rfm.head()

,recency,Frequency,monetary,T
Customer ID,,,,
12346.0,325 days 02:49:00,1,315.120,325 days 02:49:00
12347.0,1 days 20:58:00,7,4310.000,366 days 21:53:00
12348.0,74 days 23:37:00,4,1771.905,357 days 17:41:00
12349.0,18 days 02:59:00,1,1491.970,18 days 02:59:00
12350.0,309 days 20:49:00,1,331.585,309 days 20:49:00


In [194]:
# rfm["recency"] = pd.to_datetime(rfm["recency"], format='%Y-%m-%d')
rfm["recency"] = rfm["recency"].dt.days
rfm["T"] = rfm["T"].dt.days


In [195]:
rfm = rfm.sort_values(by="monetary", ascending=False)
rfm.head()


,recency,Frequency,monetary,T
Customer ID,,,,
14646.0,1,74,266376.720,354
18102.0,0,55,232526.890,366
14911.0,0,201,139053.350,372
17450.0,7,46,132623.920,367
12415.0,23,20,120312.225,337


In [196]:
rfm["RecencyScore"] = pd.qcut(rfm["recency"], 5, labels = [5, 4 , 3, 2, 1])
rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])
rfm["MonetaryScore"] = pd.qcut(rfm['monetary'], 5, labels = [1, 2, 3, 4, 5])

In [197]:
rfm.tail()

,recency,Frequency,monetary,T,RecencyScore,FrequencyScore,MonetaryScore
Customer ID,,,,,,,
17956.0,248,1,12.75,248,1,2,1
16454.0,44,2,6.90,63,3,3,1
14792.0,63,1,6.20,63,3,2,1
16738.0,297,1,3.75,297,1,2,1
13256.0,13,1,0.00,13,4,2,1


In [198]:
rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) + 
                     rfm['FrequencyScore'].astype(str) + 
                     rfm['MonetaryScore'].astype(str))
rfm.head()

,recency,Frequency,monetary,T,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,,
14646.0,1,74,266376.720,354,5,5,5,555
18102.0,0,55,232526.890,366,5,5,5,555
14911.0,0,201,139053.350,372,5,5,5,555
17450.0,7,46,132623.920,367,5,5,5,555
12415.0,23,20,120312.225,337,4,5,5,455


In [199]:
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Can\'t Loose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm["Segment"] = rfm["Segment"].replace(seg_map, regex=True)

In [213]:
rfm.head()

,recency,Frequency,monetary,T,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,,
14646.0,1,74,266376.720,354,5,5,5,555,Champions
18102.0,0,55,232526.890,366,5,5,5,555,Champions
14911.0,0,201,139053.350,372,5,5,5,555,Champions
17450.0,7,46,132623.920,367,5,5,5,555,Champions
12415.0,23,20,120312.225,337,4,5,5,455,Loyal Customers
